# CGPO Colab Backend

This notebook runs the full Cognitive Graph Portfolio Optimizer backend on Google Colab's **Tesla T4 GPU** and exposes it to your local frontend using **ngrok**.

## Instructions
1. Run all cells.
2. Enter your ngrok authtoken when prompted.
3. Copy the public URL (e.g., `https://xxxx.ngrok-free.app`).
4. Add it to your local `.env.local` file: `NEXT_PUBLIC_BACKEND_URL=https://xxxx.ngrok-free.app`

In [ ]:
# Step 1: Check GPU availability
import torch
if torch.cuda.is_available():
    print(f"Accelerated Backend Running on: {torch.cuda.get_device_name(0)}")
else:
    print("WARNING: Running on CPU. Go to Runtime -> Change runtime type -> Hardware accelerator -> GPU")

In [ ]:
# Step 2: Install dependencies
!pip install -q fastapi uvicorn pyngrok nest_asyncio
!pip install -q yfinance pandas numpy torch torch-geometric textblob ta gymnasium

In [ ]:
# Step 3: Setup Codebase
# Option A: Clone from GitHub (Recommended)
!git clone https://github.com/Vru-Jain/CGPO-Project.git

# IMPORTANT: Move into the backend directory so main.py is found
import os
try:
    os.chdir("CGPO-Project/backend")
    print(f"Current working directory: {os.getcwd()}")
except FileNotFoundError:
    print("Could not find backend directory. Did the git clone succeed?")
    # Fallback if manual upload
    if os.path.exists("/content/backend"):
        os.chdir("/content/backend")
        print(f"Changed to manual backend dir: {os.getcwd()}")

In [ ]:
# Step 4: Configure ngrok
import getpass
from pyngrok import ngrok, conf

print("Enter your ngrok authtoken (get it from https://dashboard.ngrok.com/get-started/your-authtoken):")
token = getpass.getpass()
conf.get_default().auth_token = token

# Open a HTTP tunnel on port 8000
public_url = ngrok.connect(8000).public_url
print(f"\n🚀 Your Backend URL is: {public_url}")
print(f"👉 Add this to your local frontend .env.local: NEXT_PUBLIC_BACKEND_URL={public_url}")

In [ ]:
# Step 5: Run Backend
import uvicorn
import asyncio
import sys

# Ensure current directory is in python path
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

# Fix for 'asyncio.run() cannot be called from a running event loop'
config = uvicorn.Config("main:app", host="0.0.0.0", port=8000, reload=False)
server = uvicorn.Server(config)

print("Starting server... (Stop cell to shut down)")
await server.serve()